<a href="https://colab.research.google.com/github/Wang-Kevin900/DataVisualization/blob/main/EsercizioReti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data

Importing the various library needed to the program, importing the txt sheet where all the data are (format: date - hour - value) and lastly creating all the needed variable

In [ ]:
from google.colab import files
from datetime import datetime

file_data = files.upload()
valori = list(file_data.values());

date_list = []
value_list = []
date_format = 10
hour_format = 8

string_of_data = ""

# Trasform Data

Rearrange all the data in sets of tuples 

In [ ]:
for i in valori[0]:
  if i == 13:
    pass
  elif i != 10:
    string_of_data = string_of_data + chr(i);
  else:
    date_list.append(string_of_data[0:date_format+1+hour_format])
    value_list.append(string_of_data[date_format+1+hour_format+1:])
    string_of_data = ""

Convert the value from string to float, also replace the "," in "." so that the casting is possible

In [ ]:
list_of_float_value = []
for item in value_list:
    value = item.replace(",", ".")
    list_of_float_value.append(float(value))

value_list = list_of_float_value

Convert the value from string to datetime format


In [ ]:
from datetime import datetime

list_of_date_value = []
for item in date_list:
    list_of_date_value.append(datetime.strptime(item, '%d-%m-%Y %H:%M:%S'))

date_list = list_of_date_value

Convert all the lists (date list and value list) in Pandas DataFrame, then, visualize the list created

In [ ]:
import pandas as pd

data = pd.DataFrame({'date': date_list,
                     'value': value_list})

data

# Draw Chart

Draw the Chart using the Bokeh library

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.tools import *
from IPython.display import *

output_notebook()

source = ColumnDataSource(data)

TOOLS="pan,wheel_zoom,reset,hover,poly_select,box_select"
p = figure(plot_width=1200, plot_height=400, tools=TOOLS)

# add both a line and circles on the same plot
p.line(x='date', y='value', line_width=2, source=source)
p.circle(x='date', y='value', fill_color="white", size=5, source=source)

array = ColumnDataSource(data)

select_tool = p.select(dict(type=BoxSelectTool))[0]
source.selected.js_on_change("indices", CustomJS(args=dict(p=p, source=source, array=array), code="""
        //print the selected data in the coms of the browser
        console.log(source.selected.indices);
        
        //Calculus.....
        """
))

show(p)

# Other Library used to visualize data


Test of creating the chart using various libraries...


Draw everything in a chart using the altair library

In [ ]:
import altair as alt

brush = alt.selection_interval(encodings=['x'])

color = alt.condition(brush, alt.value('blue'), alt.value('lightgray'))

chart = alt.Chart(data).mark_line(point={
    "filled": False,
    "fill": 'white'
    }).encode(
        y=alt.Y('value', sort=alt.EncodingSortField('value', order='descending')),
        x='date',
        color=color
).add_selection(
    brush
)

chart 

Draw everything in a chart using the Plotly library

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Scatter(x=date_list, y=list_of_float_value, mode="markers+lines"))

scatter = fig.data[0]

def update_point(trace, points, selector):
  print("Sleeping")
scatter.on_click(update_point)

fig.show(renderer="colab")